## A.1 Libraries

In [12]:
from fredapi import Fred
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
idx = pd.IndexSlice

In [13]:
api_file = open('api_bea.txt')
api_code = "".join(api_file.readlines())

# 1. California GDP--BEA

## 1.A Create function

In [14]:
def target_url(method, dataset, table_name, geofips, linecode,  year, format):
    api = api_code
    base = 'http://apps.bea.gov/api/data?&UserID='
    base_api = base+api

    method = '&method='+ method
    data_set = '&datasetname='+ dataset
    line_geo_year = '&GeoFIPS='+geofips+'&LineCode='+linecode+'&Year='+year

    table = '&TableName='+ table_name
    result_format = '&ResultFormat='+ format

    final_api = base_api + method + data_set + table + line_geo_year + result_format

    request = requests.get(final_api)
    bs = BeautifulSoup(request.text, 'xml')
    return bs

## 1.1 Download data

In [15]:
regional = target_url(method = "GetData", 
                      dataset = 'Regional', 
                      table_name = 'SQGDP9', 
                      geofips = 'STATE', 
                      linecode = '1',
                      year = 'ALL', 
                      format = 'XML')

## 1.2 Format data to dictionary

In [16]:
values, date, state = [],[],[]
for row in regional.find_all('Data'):  
    values.append(row.get('DataValue'))
    date.append(row.get('TimePeriod'))
    state.append(row.get('GeoName'))
data_dict = {'value':values, 'date':date, 'state':state}

## 1.3 Convert to df

In [17]:
df = pd.DataFrame.from_dict(data_dict)
df.value = [float(x.replace(',','')) for x in df.value]
df.date = [pd.to_datetime(x) for x in df.date]
df.head(2)

,value,date,state
0,14767846.0,2005-01-01,United States
1,14839707.0,2005-04-01,United States


# 2. California data

## 2.1 Subset

In [19]:
mask = df.state == 'California'
df_gdp_ca = df.loc[mask, :]
df_gdp_ca.set_index('date', inplace=True)
df_gdp_ca = df_gdp_ca.drop('state',axis=1).rename(columns={'value':'GDP_CA'})
df_gdp_ca.tail(3)

,GDP_CA
date,
2021-01-01,2799129.2
2021-04-01,2860601.9
2021-07-01,2879898.7
